# Proyek Analisis Sentimen warna
Tujuan dari proyek ini adalah untuk membuat sebuah model yang dapat memprediksi apakah warna yang dapat direkomendasikan dari inputan user, untuk warna yang ada berjumlah 42 karena dataset ini dibuat oleh saya sendiri dan dibantu juga dari chatGPT. Dataset tersebut yang berisi kumpulan kata yang labelnya adalah sebuah warna.

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

## Mempersiapkan Dataset

In [ ]:
# Baca dataset
dataset = pd.read_csv("/content/drive/MyDrive/Dataset/color_dataset.csv")

In [ ]:
dataset.head()

,text,warna,hex
0,aku ingin warna yang menenangkan,biru muda,#87CEEB
1,pilih warna yang membuatku rileks,hijau pucat,#98FB98
2,aku suka warna-warna yang cerah,kuning terang,#FFFF00
3,cari warna yang terlihat elegan,ungu tua,#800080
4,warna yang memberikan kesan alami,hijau daun,#008000


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    224 non-null    object
 1   warna   224 non-null    object
 2   hex     224 non-null    object
dtypes: object(3)
memory usage: 5.4+ KB


Berdasarkan keterangan di atas diketahui bahwa dataset yang digunakan terdiri dari 225 data dan tidak terdapat *missing value*.

## Preprocessing Data

Mengubah seluruh text kedalam bentuk lowercase

In [ ]:
dataset['text'] = dataset['text'].str.lower()
dataset.head()

,text,warna,hex
0,aku ingin warna yang menenangkan,biru muda,#87CEEB
1,pilih warna yang membuatku rileks,hijau pucat,#98FB98
2,aku suka warna-warna yang cerah,kuning terang,#FFFF00
3,cari warna yang terlihat elegan,ungu tua,#800080
4,warna yang memberikan kesan alami,hijau daun,#008000


Menghilangkan stopwords

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_word = set(stopwords.words('indonesian'))
dataset['text'] = dataset['text'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop_word)]))
dataset.head()

,text,warna,hex
0,warna menenangkan,biru muda,#87CEEB
1,pilih warna membuatku rileks,hijau pucat,#98FB98
2,suka warna-warna cerah,kuning terang,#FFFF00
3,cari warna elegan,ungu tua,#800080
4,warna kesan alami,hijau daun,#008000


In [ ]:
# Bagi dataset menjadi data fitur (X) dan label (y)
X = dataset["text"]
y = dataset["warna"]

In [ ]:
# Encoding label dengan one-hot encoding
labels = pd.get_dummies(y)

In [ ]:
# Bagi dataset menjadi data pelatihan dan data pengujian
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=123)

In [ ]:
# Melihat jumlah data pada data train dan test
print(X_train.shape)
print(X_test.shape)

(179,)
(45,)


Membuat tokenisasi

In [ ]:
filt = '!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ' # Untuk menghilangkan symbols
tokenizer = Tokenizer(num_words=2000, oov_token="<OOV>", filters=filt)
tokenizer.fit_on_texts(X_train)

In [ ]:
word_index = tokenizer.word_index
print(len(word_index))

43


Membuat sequences dan melakukan padding

In [ ]:
train_sekuens = tokenizer.texts_to_sequences(X_train)
test_sekuens = tokenizer.texts_to_sequences(X_test)


train_padded = pad_sequences(train_sekuens, 
                             maxlen=20,
                             padding='post',
                             truncating='post')
test_padded = pad_sequences(test_sekuens,
                            maxlen=20,
                            padding='post',
                            truncating='post')

In [ ]:
train_padded.shape

(179, 20)

In [ ]:
test_padded.shape

(45, 20)

## Implementasi model dengan arsitektur LSTM

In [ ]:
# Membangun model
model = Sequential()
model.add(Embedding(2000, 100, input_length=20))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(labels.shape[1], activation="softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 100)           200000    
                                                                 
 lstm (LSTM)                 (None, 64)                42240     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 42)                2730      
                                                                 
Total params: 249,130
Trainable params: 249,130
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

Latih model

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = {}):
    if(logs.get('val_accuracy') > 0.90 and logs.get('accuracy') > 0.90):
      print("\nPELATIHAN BERHENTI, AKURASI MODEL SUDAH LEBIH DARI 90%!")
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
# Melatih model
model.fit(train_padded, y_train, 
          epochs=500, 
          validation_data=(test_padded, y_test),
          verbose=2,
          callbacks=[callbacks])

Epoch 1/500
6/6 - 5s - loss: 3.7380 - accuracy: 0.0112 - val_loss: 3.7311 - val_accuracy: 0.0000e+00 - 5s/epoch - 813ms/step
Epoch 2/500
6/6 - 0s - loss: 3.7243 - accuracy: 0.0168 - val_loss: 3.7240 - val_accuracy: 0.0000e+00 - 294ms/epoch - 49ms/step
Epoch 3/500
6/6 - 0s - loss: 3.7144 - accuracy: 0.0000e+00 - val_loss: 3.7111 - val_accuracy: 0.0000e+00 - 269ms/epoch - 45ms/step
Epoch 4/500
6/6 - 0s - loss: 3.7001 - accuracy: 0.0168 - val_loss: 3.6962 - val_accuracy: 0.0000e+00 - 272ms/epoch - 45ms/step
Epoch 5/500
6/6 - 0s - loss: 3.6711 - accuracy: 0.0223 - val_loss: 3.6760 - val_accuracy: 0.0000e+00 - 265ms/epoch - 44ms/step
Epoch 6/500
6/6 - 0s - loss: 3.6580 - accuracy: 0.0559 - val_loss: 3.6574 - val_accuracy: 0.0889 - 294ms/epoch - 49ms/step
Epoch 7/500
6/6 - 0s - loss: 3.6363 - accuracy: 0.0670 - val_loss: 3.6475 - val_accuracy: 0.0222 - 264ms/epoch - 44ms/step
Epoch 8/500
6/6 - 1s - loss: 3.6045 - accuracy: 0.0559 - val_loss: 3.6414 - val_accuracy: 0.0222 - 504ms/epoch - 84ms

In [ ]:
# Evaluasi model
loss, accuracy = model.evaluate(test_padded, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)

2/2 [==============================] - 0s 12ms/step - loss: 0.4287 - accuracy: 0.8889
Loss: 0.42867469787597656
Accuracy: 0.8888888955116272


In [ ]:
color_dict = {
    0: 'Biru Muda',
    1: 'Hijau Pucat',
    2: 'Kuning Terang',
    3: 'Ungu Tua',
    4: 'Hijau Daun',
    5: 'Mint',
    6: 'Biru Laut',
    7: 'Hijau Muda',
    8: 'Emas',
    9: 'Merah Jambu',
    10: 'Abu-abu',
    11: 'Merah Cerah',
    12: 'Oranye',
    13: 'Hitam',
    14: 'Ungu Muda',
    15: 'Kuning Cerah',
    16: 'Merah Maroon',
    17: 'Putih',
    18: 'Cokelat',
    19: 'Kuning Ceria',
    20: 'Merah Muda',
    21: 'Oranye Terang',
    22: 'Abu-abu Metalik',
    23: 'Hijau Terang',
    24: 'Biru Royal',
    25: 'Hijau Daun Muda',
    26: 'Kuning Kunyit dan Emas',
    27: 'Merah Anggur',
    28: 'Emerald, Biru Laut',
    29: 'Biru Navy',
    30: 'Biru Pastel',
    31: 'Hijau Zamrud',
    32: 'Kuning Lemon',
    33: 'Merah Darah',
    34: 'Lavender',
    35: 'Ungu Lavender',
    36: 'Kuning Matahari',
    37: 'Merah Menyala',
    38: 'Jingga',
    39: 'Biru Langit',
    40: 'Mawar',
    41: 'Silver'
}

# Contoh prediksi warna berdasarkan masukan pengguna
def predict_color(input_text):
    input_text = tokenizer.texts_to_sequences([input_text])
    input_text = pad_sequences(input_text, maxlen=20, padding="post")
    predicted_probabilities = model.predict(input_text)[0]
    predicted_label = np.argmax(predicted_probabilities)
    predicted_color = color_dict[predicted_label]
    return "Saya merekomendasikan warna " + predicted_color

In [ ]:
# Contoh penggunaan model untuk prediksi
input_text = "kasih aku warna yang bikin aku jadi keliatan kalem"
predicted_color = predict_color(input_text)
print("Prediksi warna:", predicted_color)

1/1 [==============================] - 0s 29ms/step
Prediksi warna: Saya merekomendasikan warna Emerald, Biru Laut


## Save Model

In [ ]:
# Simpan model menggunakan format Keras (.h5)
model.save('color_sentiment.h5')

# Konversi model ke format TFLite (.tflite)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False  # Nonaktifkan penurunan operasi tensor list
tflite_model = converter.convert()

# Simpan model TFLite ke file
with tf.io.gfile.GFile('color_sentiment.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model telah disimpan.")

Model telah disimpan.
